<a href="https://colab.research.google.com/github/JMartinArocha/MasterBigData/blob/main/3_2_Iris_NaiveBayes_AllClasifiers%2BcrossValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
# importación de datos
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

iris = sns.load_dataset('iris')

# 1. Creamos un array con los distintos clasificadores
clasificadores = [MultinomialNB(), GaussianNB(),ComplementNB(),BernoulliNB(), CategoricalNB()]
print('Sin Cross Validation')
# Ejecutamos con cada clasificador
for clasificador in clasificadores:
    # separamos datos de entrada y salida
    X_iris = iris.drop('species', axis=1)
    y_iris = iris['species']
    # separamos train y test
    Xtrain, Xtest, ytrain, ytest = train_test_split(X_iris, y_iris, test_size=0.33, random_state=1)

    nombre_clasificador = clasificador.__class__.__name__
    # 2. instanciamos el modelo
    model = clasificador
    # 3. Entrenamiento con los datos
    model.fit(Xtrain, ytrain)
    # 4. Predicción con nuevos datos
    y_model = model.predict(Xtest)

    # 5 evaluación
    # Compara si los resultados obtenidos por el modelo (y_model) coinciden con los datos y_test esperados
    score = accuracy_score(ytest, y_model) # Precisión del modelo
    print(f"Modelo: {nombre_clasificador} Score: {score}")

    # Y finalmente visualizamos la Matriz de Confusión, para lo que se crea la siguiente función, que se llamará más adelante

    # y_true : dataframe -> Los valores de las clases que son ciertos (test)
    # y_pred : ndarray   -> Los valores calculados de las clases después de realizar la predicción
    # class  : ndarray   -> Los nombres de las clases/valores objetivos

    def plot_confusion_matrix(y_true, y_pred, classes,
                            normalize=False,
                            title=None,
                            cmap=plt.cm.Blues,
                            titleSup=None):
        """
        This function prints and plots the confusion matrix.
        Normalization can be applied by setting `normalize=True`.
        """
        if not title:
            if normalize:
                title = 'Normalized confusion matrix'
            else:
                title = 'Confusion matrix, without normalization'

        # Compute confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        # Only use the labels that appear in the data
        classes = classes[unique_labels(y_true, y_pred)]
        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            # print("Normalized confusion matrix")
        # else:
            # print('Confusion matrix, without normalization')

        # print(cm)

        fig, ax = plt.subplots()
        im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
        ax.figure.colorbar(im, ax=ax)
        # We want to show all ticks...
        ax.set(xticks=np.arange(cm.shape[1]),
            yticks=np.arange(cm.shape[0]),
            # ... and label them with the respective list entries
            xticklabels=classes, yticklabels=classes,
            title=title,
            ylabel='True label',
            xlabel='Predicted label')

        # Rotate the tick labels and set their alignment.
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
                rotation_mode="anchor")

        # Loop over data dimensions and create text annotations.
        fmt = '.2f' if normalize else 'd'
        thresh = cm.max() / 2.
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax.text(j, i, format(cm[i, j], fmt),
                        ha="center", va="center",
                        color="white" if cm[i, j] > thresh else "black")
        fig.tight_layout()
        fig.suptitle(titleSup, fontsize=16, y=1, ha='center')
        return ax

    # Transformamos variables categóricas a valores numéricos/enteros para llamar a la función que pinta la matriz de confusión plot_confusion_matrix()
    ytest_df= ytest.to_frame()
    ytest_df['species'].replace(['setosa', 'versicolor', 'virginica'],[0, 1, 2], inplace=True)

    # Transformamos variables categóricas a array para poder llamar a la función que pinta la matriz de confusión plot_confusion_matrix()
    # Transformamos el resultado de la predicción(array) a un dataframe para transformar los valores categóricos en numéricos
    y_model_df= pd.DataFrame(y_model, columns = ['species'])
    y_model_df['species'].replace(['setosa', 'versicolor', 'virginica'],[0, 1, 2], inplace=True)
    # Y volvemos a transformar el dataframe a un array, que es el tipo de dato que espera la función plot_confusion_matrix()
    y_model_array = y_model_df['species'].to_numpy()

    # Creamos este array porque es el parámetro con las clases que espera la función
    # clases_iris = np.array(['setosa', 'versicolor', 'virginica'])
    # nombre_clasificador = f"{clasificador.__class__.__name__} Score: {score}"
    # plot_confusion_matrix(ytest_df['species'], y_model_array, classes=clases_iris, normalize=False, title='Normalized confusion matrix', titleSup=nombre_clasificador)

Sin Cross Validation
Modelo: MultinomialNB Score: 0.64
Modelo: GaussianNB Score: 0.94
Modelo: ComplementNB Score: 0.62
Modelo: BernoulliNB Score: 0.28
Modelo: CategoricalNB Score: 0.98


In [54]:
# Añadimos cross validation
crossValidation = KFold(n_splits = 5, shuffle = True) # shuffle = False si hay dimensión temporal

total_scores = []
for clasificador in clasificadores:
  Xtrain, Xtest, ytrain, ytest = train_test_split(X_iris, y_iris, test_size=0.33, random_state=1)

  model = clasificador
  fold_accuracy = []
  for train_fold, test_fold in crossValidation.split(Xtrain):
      # División train test aleatoria
      f_train_x = Xtrain.iloc[train_fold] # Extrae la información (iloc), atendiendo a los indices obtenidos por CrossValidation
      f_train_y = ytrain.iloc[train_fold]
      # entrenamiento
      model.fit(f_train_x, f_train_y)
      # Realizamos la predicción (Final evaluation) y guardamos la precisión para calcular la media posteriormente
      y_pred = model.predict(Xtrain.iloc[test_fold])
      # evaluación del modelo
      acc = accuracy_score(ytrain.iloc[test_fold], y_pred)
      fold_accuracy.append(acc)

  total_scores.append(sum(fold_accuracy)/len(fold_accuracy))
print('Con Cross Validation')
for i in range(len(clasificadores)):
    nombre_clasificador = f"{clasificadores[i].__class__.__name__}"
    print(f"Modelo: {nombre_clasificador} Score: {total_scores[i]}")

Con Cross Validation
Modelo: MultinomialNB Score: 0.8099999999999999
Modelo: GaussianNB Score: 0.93
Modelo: ComplementNB Score: 0.6900000000000001
Modelo: BernoulliNB Score: 0.19
Modelo: CategoricalNB Score: 0.9100000000000001
